# 🏭 Generador de Tablas para el Método Cascada
**Asignatura:** Automatización Industrial
**Lenguaje:** Python

Esta herramienta automatiza la creación de tablas lógicas para circuitos neumáticos. Solo necesitas ingresar la secuencia de movimientos y el algoritmo se encargará de dividir los grupos y asignar las señales de control.

In [1]:
import pandas as pd
import re
from IPython.display import display, HTML, Markdown

def analizar_secuencia_cascada(secuencia: str):
    """
    Analiza una secuencia neumática (Método Cascada) y genera tablas de lógica de control.

    Args:
        secuencia (str): Cadena con la secuencia (ej. "A+ B- C+[2s]").
    """
    # Limpieza inicial de la cadena
    secuencia = secuencia.strip().upper()
    print(f"🔄 Procesando secuencia: {secuencia}\n")

    # --- 1. Parsing (Análisis) de la secuencia ---
    # Regex para capturar: Letra, Signo y Opcionalmente Temporizador [Xs]
    patron = r'([A-Z])([+-])(\[\d+s\])?'
    matches = re.findall(patron, secuencia)

    # Reconstruimos los movimientos en una lista limpia
    movimientos_completos = [f"{m[0]}{m[1]}{m[2]}" for m in matches] # Ej: 'C+[2s]'
    solo_movimientos = [f"{m[0]}{m[1]}" for m in matches]            # Ej: 'C+'

    if not matches:
        display(Markdown("### ⚠️ Error: La secuencia no tiene el formato correcto."))
        return

    # --- 2. Algoritmo de División de Grupos (Cascada) ---
    grupos = []
    grupo_actual = []
    letras_en_grupo = set()

    for i, mov in enumerate(solo_movimientos):
        letra = mov[0] # El pistón (A, B, C...)

        # Regla Cascada: Si la letra ya está en el grupo, se cierra el grupo actual
        if letra in letras_en_grupo:
            grupos.append(grupo_actual)
            grupo_actual = [movimientos_completos[i]]
            letras_en_grupo = {letra}
        else:
            grupo_actual.append(movimientos_completos[i])
            letras_en_grupo.add(letra)

    grupos.append(grupo_actual) # Añadir el último grupo pendiente

    # --- 3. Generación de Datos para Tablas ---

    # Preparar datos para Tabla 1 (Desarrollo)
    data_desarrollo = []
    paso = 1

    # Lista plana de todos los finales de carrera que se activan
    # Si es A+, activa a1. Si es A-, activa a0.
    todos_fc = []
    for mov in solo_movimientos:
        letra = mov[0].lower()
        fc = f"{letra}1" if '+' in mov else f"{letra}0"
        todos_fc.append(fc)

    idx_global = 0
    for i, grupo in enumerate(grupos):
        for mov in grupo:
            # Determinamos qué FC se activa AL FINAL de este movimiento
            fc_actuado = todos_fc[idx_global]

            data_desarrollo.append({
                'Paso': paso,
                'Grupo': f"Grupo {i + 1}",
                'Movimiento': mov,
                'FC Actuado': fc_actuado
            })
            paso += 1
            idx_global += 1

    df_desarrollo = pd.DataFrame(data_desarrollo)

    # Preparar datos para Tabla 2 (Control de Grupos)
    num_grupos = len(grupos)
    data_grupos = []

    # Lógica: Grupo 1 se activa con el último FC del último Grupo
    ultimo_fc_secuencia = todos_fc[-1]
    data_grupos.append({
        'Grupo': 'Grupo 1',
        'Se Activa Con': f"START AND Grupo {num_grupos} AND {ultimo_fc_secuencia}"
    })

    # Lógica: Grupos siguientes (G(n) se activa con G(n-1) + último FC de G(n-1))
    # Necesitamos saber cuál fue el último movimiento del grupo anterior
    conteo_movimientos = 0
    for i in range(num_grupos - 1):
        len_grupo_anterior = len(grupos[i])
        # El índice del último movimiento del grupo anterior en la lista global 'todos_fc'
        idx_ultimo_fc_anterior = conteo_movimientos + len_grupo_anterior - 1
        fc_activador = todos_fc[idx_ultimo_fc_anterior]

        data_grupos.append({
            'Grupo': f"Grupo {i + 2}",
            'Se Activa Con': f"Grupo {i + 1} AND {fc_activador}"
        })
        conteo_movimientos += len_grupo_anterior

    df_grupos = pd.DataFrame(data_grupos)

    # Preparar datos para Tabla 3 (Control de Movimientos)
    data_movs = []
    conteo_movimientos = 0

    for i, grupo in enumerate(grupos):
        nombre_grupo = f"Grupo {i + 1}"
        for j, mov in enumerate(grupo):
            mov_limpio = solo_movimientos[conteo_movimientos] # Sin temporizador para lógica
            condicion = ""

            if j == 0:
                # Primer movimiento del grupo: Directo del Grupo
                condicion = nombre_grupo
            else:
                # Movimiento intermedio: Grupo + FC anterior
                fc_anterior = todos_fc[conteo_movimientos - 1]
                condicion = f"{nombre_grupo} AND {fc_anterior}"

            data_movs.append({
                'Movimiento': mov_limpio, # Usamos la versión limpia para la tabla de lógica
                'Se Activa Con': condicion
            })
            conteo_movimientos += 1

    # Agrupar si un movimiento se repite en la secuencia (ej. C+ aparece dos veces)
    df_movs_raw = pd.DataFrame(data_movs)
    df_movs = df_movs_raw.groupby('Movimiento')['Se Activa Con'].apply(lambda x: ' OR '.join(x)).reset_index()

    # Ordenar alfabéticamente para mayor claridad
    df_movs = df_movs.sort_values(by='Movimiento')


    # --- 4. Visualización Profesional ---

    # Estilos CSS para las tablas
    estilo_tabla = [
        {'selector': 'th', 'props': [('background-color', '#2c3e50'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center'), ('border', '1px solid #ddd')]},
        {'selector': 'tr:nth-of-type(even)', 'props': [('background-color', '#f2f2f2')]},
        {'selector': 'tr:hover', 'props': [('background-color', '#ddd')]}
    ]

    display(Markdown("### 📊 Resultado del Análisis"))

    display(Markdown("#### 1. Tabla de Desarrollo de la Secuencia"))
    display(df_desarrollo.style.set_table_styles(estilo_tabla).hide(axis='index'))

    display(Markdown("#### 2. Ecuaciones de Activación de Grupos (Memorias)"))
    display(df_grupos.style.set_table_styles(estilo_tabla).hide(axis='index'))

    display(Markdown("#### 3. Ecuaciones de Activación de Solenoides (Válvulas)"))
    display(df_movs.style.set_table_styles(estilo_tabla).hide(axis='index'))

    return df_desarrollo, df_grupos, df_movs

In [2]:
# --- PRUEBA CON EL EJERCICIO DEL PDF ---
# Secuencia: A- B- C+[2s] C- C+[2s] C- B+ A+

secuencia_ejercicio = "A- B- C+[2s] C- C+[2s] C- B+ A+"

# Ejecutamos la función
df1, df2, df3 = analizar_secuencia_cascada(secuencia_ejercicio)

🔄 Procesando secuencia: A- B- C+[2S] C- C+[2S] C- B+ A+



### 📊 Resultado del Análisis

#### 1. Tabla de Desarrollo de la Secuencia

Paso,Grupo,Movimiento,FC Actuado
1,Grupo 1,A-,a0
2,Grupo 1,B-,b0
3,Grupo 1,C+,c1
4,Grupo 2,C-,c0
5,Grupo 3,C+,c1
6,Grupo 4,C-,c0
7,Grupo 4,B+,b1
8,Grupo 4,A+,a1


#### 2. Ecuaciones de Activación de Grupos (Memorias)

Grupo,Se Activa Con
Grupo 1,START AND Grupo 4 AND a1
Grupo 2,Grupo 1 AND c1
Grupo 3,Grupo 2 AND c0
Grupo 4,Grupo 3 AND c1


#### 3. Ecuaciones de Activación de Solenoides (Válvulas)

Movimiento,Se Activa Con
A+,Grupo 4 AND b1
A-,Grupo 1
B+,Grupo 4 AND c0
B-,Grupo 1 AND a0
C+,Grupo 1 AND b0 OR Grupo 3
C-,Grupo 2 OR Grupo 4
